<a href="https://colab.research.google.com/github/Bourbon-Rye/Baesian-Cropability/blob/main/PilipiNuts_2023_Baesian_Cropability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [31]:
# @Libraries
import numpy as np
import pandas as pd
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import json
import re
import functools

from pathlib import Path
from sklearn import preprocessing
from plotly.subplots import make_subplots
from scipy.stats import chi2_contingency
from sklearn.impute import SimpleImputer

In [32]:
# @title Corrections
with open('datasets/region_provinces.json') as jsonfile:
    regions_provinces = json.load(jsonfile)['PHILIPPINES']
regions_provinces = {key.lower():regions_provinces[key] for key in regions_provinces}
regions = regions_provinces.keys()
provinces = set([item.lower() for key in regions for item in regions_provinces[key]])
regions = set(regions)

# NOTE: Fix for bad regions, thanks PSA
bad_regions = ['AONCR', 'BARMM', 'CAR', 'MIMAROPA', 'NCR',
           'Region 1', 'Region 2', 'Region 3', 'Region 4A',
           'Region 5', 'Region 6', 'Region 7', 'Region 8',
           'Region 9', 'Region 10', 'Region 11', 'Reg12', 'CARAGA']
region_mapping = {bad.lower():good.lower() for (bad,good) in zip(bad_regions, regions_provinces.keys())}
corrections = {
    "AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)": "bangsamoro autonomous region in muslim mindanao (barmm)",
    "autonomous region in muslim mindanao (armm)": "bangsamoro autonomous region in muslim mindanao (barmm)",
    "mimaropa region": "mimaropa region (mimaropa)"
}
region_mapping.update(corrections)

temp = [key.split('(')[1].rstrip(')') for key in regions_provinces]
region_short_to_short_long = {bad:good for (bad,good) in zip(temp, regions_provinces.keys())}

In [30]:
# @title Utilities
def read_csv_to_df(csvfile: Path, comment_symbol='#') -> pd.DataFrame:
    """Read CSV to DataFrame with comment validation.
        Allows comment lines in CSVs where line[0] == comment_symbol.
        Also removes newlines.
        Does not recognize comment_symbol anywhere else. 
    """
    tempfile = Path('temp.csv')
    with open(csvfile, 'r') as csv, open(tempfile, 'w+') as temp:
        lines = csv.readlines()
        for line in lines:
            if line[0] != comment_symbol:
                temp.write(line)
    return pd.read_csv(tempfile)

def is_region(x: str, regions=regions) -> bool:
    x = x.strip(' .')
    x = x.lower()
    if x == "cagayan":
        return False
    for region in regions:
        if x in region:
            return True
    else:
        return False
    
def is_province(x: str, provinces=provinces) -> bool:
    x = x.strip(' .')
    x = x.lower()
    for province in provinces:
        if x in province:
            return True
    else:
        return False

def get_quarter_columns(df: pd.DataFrame, year_range: range, period_idx: int):
    """Assumes contiguous period (Year Month) columns and that columns before period_idx are ID columns"""
    df_quarter = df.iloc[:, :period_idx]
    for year in year_range:
        for q in range(0, 12, 3):
            df_quarter[f"{year} Q{q//3+1}"] = df.filter(regex=str(year), axis=1).iloc[:, q:q+3].mean(axis=1)
    return df_quarter

def get_annual_columns(df: pd.DataFrame, year_range: range, period_idx: int):
    """Assumes contiguous period (Year Month|Quarter) columns and that columns before period_idx are ID columns
    Note that this also works with Quarters"""
    df_annual = df.iloc[:, :period_idx]
    for year in year_range:
        df_annual[f"{year}"] = df.filter(regex=str(year), axis=1).mean(axis=1)
    return df_annual

def swap_columns(df: pd.DataFrame, col1: str, col2: str):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

def normalize(df: pd.DataFrame, col: str, minmax = True):
    """Can use mean normalization and minmax normalization."""
    tdf = df[col]
    if minmax:
        tdf = (tdf-tdf.min())/(tdf.max()-tdf.min())
    else:
        tdf = (tdf-tdf.mean())/tdf.std()
    df[col] = tdf
    return df

def drop_rows_with_zeros(df: pd.DataFrame, ref_col_idx: int, all_zeros=False):
    """Drop rows if some values are zeros, or if all values are zeros.
    Assumes contiguous reference columns, i.e. columns to use in deciding whether to drop."""
    return df[~(df.iloc[:, ref_col_idx:] == 0).all(axis=1)] if all_zeros else df[~(df.iloc[:, ref_col_idx:] == 0).any(axis=1)]

def dual_plot(df: pd.DataFrame, y1: str, y2: str,
              x_title: str, y1_title: str, y2_title: str, title_text: str):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Add traces
    fig.add_trace(
        go.Scatter(x=df.index, y=df[y1], name=y1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df[y2], name=y2),
        secondary_y=True,
    )
    # Add titles
    fig.update_layout(title_text=title_text)
    fig.update_xaxes(title_text=x_title)
    # Set y-axes titles
    fig.update_yaxes(title_text=y1_title, secondary_y=False)
    fig.update_yaxes(title_text=y2_title, secondary_y=True)

    fig.show()

def move_column(df: pd.DataFrame, col: str, new_idx: int):
    """This is an inplace method."""
    df.insert(new_idx, col, df.pop(col))
    
def preprocess_baesians_1(df: pd.DataFrame, commodity: str, rename_to: str, melt_value: str|None, regional=True):
    """Assumes Geolocation | Commodity | Period columns.
    Filters to regional if regional=True, else filters to provincial.
    """
    # df.dropna(inplace=True)
    df["Geolocation"] = df["Geolocation"].str.lstrip(".").str.lower()
    df["Geolocation"] = df["Geolocation"].replace(region_mapping)
    df = df[df["Geolocation"].apply(is_region)] if regional else df[df["Geolocation"].apply(is_province)]
    df = df[df["Commodity"] == commodity]
    df["Commodity"] = rename_to # Rename everything in Commodity to Rice
    if melt_value:
        df = df.melt(id_vars=["Geolocation", "Commodity"], value_vars=df.columns[2:], var_name="Period", value_name=melt_value)
    return df

def preprocess_baesians_2(df: pd.DataFrame, commodity_map: dict, melt_value: str|None):
    """Assumes Geolocation | Commodity | Period ... columns.
    Filters to raw commodity name (key in commodity_map), and then renames to standard commodity_map[key]
    Ex. {"RICE, REGULAR-MILLED, 1 KG" : "Rice"}
    """
    df["Geolocation"] = df["Geolocation"].str.lstrip(".").str.lower()
    df["Geolocation"] = df["Geolocation"].replace(region_mapping)
    df = df[df["Commodity"].isin(commodity_map)]
    df.loc[:, "Commodity"] = df["Commodity"].replace(commodity_map)
    if melt_value:
        df = df.melt(id_vars=["Geolocation", "Commodity"], value_vars=df.columns[2:], var_name="Period", value_name=melt_value)
    return df

def filter_to_regions(df: pd.DataFrame):
    return df[df["Geolocation"].apply(is_region)]
    
def filter_to_provinces(df: pd.DataFrame):
    return df[df["Geolocation"].apply(is_province)]

In [33]:
# @title OTG Cleanup
# VALUE OF PRODUCTION (5/24/2024)
datapath = Path("datasets/agricultural-indicators/value-of-production/")
writepath = Path("datasets")

files = list(datapath.glob("*.csv"))
df = pd.DataFrame()
for file in files:
    with open(file) as f:
        region = f.readlines()[0].split(':')[0].lstrip("\"")
        if region == u"\ufeffTest\n":
            region = "Cordillera Administrative Region (CAR)"
        tdf = pd.read_csv(file, skiprows=1, na_values=".")
        tdf["Geolocation"] = region
        df = pd.concat([df, tdf])
df["Subsector"] = "Rice"
df.drop(columns=["Type of Valuation"], inplace=True)
df.rename({"Subsector": "Commodity"}, axis=1, inplace=True)

# Imputation of VOP df
imp = SimpleImputer(missing_values=pd.NA, strategy='mean')
imp.fit(df.iloc[:, 1:-1])
df[df.columns[1:-1]] = imp.transform(df.iloc[:, 1:-1])
df_val_of_prod = df

# ANNUAL RICE STOCKS (5/24/2024) NOTE: Perhaps try to match this with the prices as this has monthly, for national only
df = read_csv_to_df("datasets/agricultural-indicators/stocks-palay-corn_yearly_1980-2024.csv")
df.dropna(inplace=True)
df = drop_rows_with_zeros(df, ref_col_idx=2, all_zeros=True)
df = df[df["Sector"] == "Rice: Total Stock"]
df["Sector"] = "Rice"
df = pd.concat([df.iloc[:, :2], df.mean(axis=1, numeric_only=True)], axis=1)
df.rename(columns={"Sector": "Commodity", "Year": "Period", 0: "Stocks"}, inplace=True)
df["Period"] = df["Period"].astype(object)
df = df[df["Period"].isin(range(2012, 2024))]
df_stocks = df.set_index("Period", drop=True)

# Baesian Plots


Hypotheses:
- **H0.1:** There is no significant difference in the productivity of major food crops when grouped according to their crop type, geolocation, and/or market profile.
- **H0.2:** There is no significant relationship between market conditions and food crop production.

Goal: Visually and statistically assess the relationship between market indicators and crop yield indicators.

Visual tests are dual plots and scatterplots. Statistics for relationship testing: contingency tables and t-tests (?).

In [9]:
# Sample Relationship Test
data = [[207, 282, 241], [234, 242, 232]]
stat, p, dof, expected = chi2_contingency(data)
 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.10319714047309392
Independent (H0 holds true)


## Data Overview

Selected major crops:
- Rice / Palay
- Corn / Maize
- Sweet Potato / Camote
- Mongo / Monggo / Mung Beans
- Banana
- Coconut
- Onion
- Garlic
- Sugarcane

In [44]:
# Volume of Production of Selected Major Crops
df = pd.read_csv("datasets/agricultural-indicators/volume_rice-corn.csv", skiprows=2)
df = df[df["Geolocation"] == "PHILIPPINES"]
df = df.filter(regex="Commodity|Geolocation|Annual", axis=1)
df.columns = map(lambda x: x.replace(" Annual", "") if "Annual" in x else x, df.columns)
df = df.melt(id_vars=["Commodity", "Geolocation"], value_vars=df.columns[2:], var_name="Period", value_name="Volume")
fig = px.line(df, x="Period", y="Volume", color="Commodity", title='Annual Volume of Production of Rice and Corn<br>National average in metric tons').update_layout(
    xaxis_title="Period", yaxis_title = "Volume of Production")
fig.show()

df = pd.read_csv("datasets/agricultural-indicators/volume_other-crops.csv", skiprows=2)
df = df[df["Geolocation"] == "PHILIPPINES"]
df = df.filter(regex="Commodity|Geolocation|Annual", axis=1)
df.columns = map(lambda x: x.replace(" Annual", "") if "Annual" in x else x, df.columns)
df = df.melt(id_vars=["Commodity", "Geolocation"], value_vars=df.columns[2:], var_name="Period", value_name="Volume")
fig = px.line(df, x="Period", y="Volume", color="Commodity", title='Annual Volume of Production of Other Major Crops<br>National average in metric tons').update_layout(
    xaxis_title="Period", yaxis_title = "Volume of Production")
fig.show()

## Annual Analysis
Annual-National, Annual-Regional, and Annual-Provincial analysis of the commodities Rice, Corn, and Potato.

In [29]:
# @title Annual megadataset for rice and corn
# Volume of Rice and Corn
df1 = read_csv_to_df("datasets/agricultural-indicators/volume_rice-and-corn_annual.csv")
df1 = preprocess_baesians_2(df1, {"Palay": "Rice", "Corn": "Corn"}, melt_value="Volume")

# Farmgate Price of Rice and Corn
df = read_csv_to_df("datasets/prices/prices_farmgate-new-series_2010-2023.csv")
df = get_annual_columns(df, range(2012, 2024), 2)
df = preprocess_baesians_2(df, {"Palay [Paddy] Other Variety, dry (conv. to 14% mc)": "Rice",
                                  "Corngrain [Maize] Yellow, matured": "Corn"}, melt_value="Farmgate Price")

# # Retail Price of Rice and Corn
# df3 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
# tdf1 = pd.read_csv("datasets/prices/prices_retail_caraga_supplement.csv", skiprows=1)
# tdf2 = pd.read_csv("datasets/prices/prices_retail_car_supplement.csv", skiprows=1)
# df3 = pd.concat([df3, tdf1, tdf2])
# df3 = get_quarter_columns(df3, range(2012, 2024), 2)
# df3 = preprocess_baesians_1(df3, "RICE, REGULAR-MILLED, 1 KG", "Rice", melt_value="Retail Price")

df1[df1["Geolocation"].apply(is_region)]["Geolocation"].unique()

array(['cordillera administrative region (car)',
       'region i (ilocos region)', 'region ii (cagayan valley)',
       'region iii (central luzon)', 'region iv-a (calabarzon)',
       'mimaropa region (mimaropa)', 'region v (bicol region)',
       'region vi (western visayas)', 'region vii (central visayas)',
       'region viii (eastern visayas)', 'region ix (zamboanga peninsula)',
       'region x (northern mindanao)', 'region xi (davao region)',
       'region xii (soccsksargen)', 'region xiii (caraga)',
       'bangsamoro autonomous region in muslim mindanao (barmm)'],
      dtype=object)

## Quarterly-Regional Analysis

In [16]:
# @title Quarterly megadataset for rice
# Volume of Rice
df1 = read_csv_to_df("datasets/agricultural-indicators/volume_rice-and-corn_quarterly.csv")
df1 = preprocess_baesians_1(df1, "Palay", "Rice", melt_value="Volume")

# Farmgate Price of Rice
df = read_csv_to_df("datasets/prices/prices_farmgate-new-series_2010-2023.csv")
df = get_quarter_columns(df, range(2012, 2024), 2)
df = preprocess_baesians_1(df, "Palay [Paddy] Other Variety, dry (conv. to 14% mc)", "Rice", melt_value="Farmgate Price")
df.dropna(inplace=True)

# Retail Price of Rice
df3 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
tdf1 = pd.read_csv("datasets/prices/prices_retail_caraga_supplement.csv", skiprows=1)
tdf2 = pd.read_csv("datasets/prices/prices_retail_car_supplement.csv", skiprows=1)
df3 = pd.concat([df3, tdf1, tdf2])
df3 = get_quarter_columns(df3, range(2012, 2024), 2)
df3 = preprocess_baesians_1(df3, "RICE, REGULAR-MILLED, 1 KG", "Rice", melt_value="Retail Price")

# Wholesale Price of Rice
df4 = pd.read_csv("datasets/prices/prices_wholesale-new-series_2010-2023.csv",)
df4 = get_quarter_columns(df4, range(2012, 2024), 2)
df4 = preprocess_baesians_1(df4, "Well Milled Rice (WMR)", "Rice", melt_value="Wholesale Price")

# Cropyield (Area Harvested) of Rice
df5 = pd.read_csv("datasets/agricultural-indicators/area-harvested-palay-corn_quarterly-annual_2010-2023.csv", skiprows=1)
df5 = preprocess_baesians_1(df5, "Palay", "Rice", melt_value="Area Harvested")
df5 = df5[df5["Period"].str.contains("Q")]

# Consumer Price Index (All Income) per Region of Rice
df6 = pd.read_csv("datasets/price-indices-2018-based/cpi_all-income-households-by-cg-with-backcasting_1994-2023.csv")
df6 = get_quarter_columns(df6, range(2012, 2024), 2)
df6 = preprocess_baesians_1(df6, "01.1.1.12 - Rice", "Rice", melt_value="CPI All Income")

# Consumer Price Index (Bottom 30) per Region of Rice
df7 = pd.read_csv("datasets/price-indices-2018-based/cpi_bottom-30-by-cg-with-backcasting_2012-2017.csv")
df7 = preprocess_baesians_1(df7, "01.1.1.12 - Rice", "Rice", melt_value=None)
tdf = pd.read_csv("datasets/price-indices-2018-based/cpi_bottom-30-by-cg_2018-2023.csv")
tdf = preprocess_baesians_1(tdf, "01.1.1.12 - Rice", "Rice", melt_value=None)
df7 = pd.merge(df7, tdf, on=["Geolocation", "Commodity"])
df7 = get_quarter_columns(df7, range(2012, 2024), 2)
df7 = preprocess_baesians_1(df7, "Rice", "Rice", melt_value="CPI Bottom 30")

# Merge all dfs into a single df NOTE: geolocation must be in lower case
# NOTE: barmm is removed because it has null values, unfortunately
dfs = [df1, df, df3, df4, df5, df6, df7]
df = functools.reduce(lambda left, right: pd.merge(left, right), dfs)
move_column(df, "Area Harvested", 4)

# ------------------------------------

# Value of Production of Rice (Annual)
df1 = df_val_of_prod
col = df1.pop("Geolocation")
df1.insert(0, col.name, col)
df1 = preprocess_baesians_1(df1, "Rice", "Rice", melt_value="Value of Production")

tdf = df.iloc[:,:3]
tdf["Period"] = df["Period"].str.replace(" Q\d", "", regex=True)
df["Value of Production"] = pd.merge(tdf, df1)["Value of Production"]

# Stocks of Rice (Annual)
df1 = df_stocks
tdf = df["Period"].str.replace(" Q\d", "", regex=True)
distributed = []
for period in tdf:
    distributed.append(df1.loc[int(period), "Stocks"])
df.insert(5, "Stocks", pd.Series(distributed))

# for x in ["Volume", "Area Harvested", "Stocks"]:
#     for y in ["Retail Price", "Wholesale Price", "Farmgate Price", "CPI All Income", "CPI Bottom 30", "Value of Production"]:
#         fig = px.scatter(df, x=x, y=y, color="Geolocation")
#         # fig.show()

# Correlation Matrix
contingency = df.iloc[:, 3:].corr(method="spearman")
cont_1 = contingency.iloc[3:, 0:3]
fig = px.imshow(cont_1, text_auto=True)
fig.show()

In [ ]:
# Statistics for 
# Import the necessary libraries:
from scipy import stats
from termcolor import colored

sample_size = 500
# Randomly sample 500 rows from the Quarterly-Regional Market-Farm dataset:
df = df.sample(n=sample_size)

# Filter the dataset for the two variables we want to compare:
for x in ["Volume", "Area Harvested", "Stocks"]:
    for y in ["Retail Price", "Wholesale Price", "Farmgate Price", "CPI All Income", "CPI Bottom 30", "Value of Production"]:
        farm = df[x]
        market = df[y]

        # Perform the t-test:
        t_stat, p_value = stats.ttest_ind(farm, market)

        # Interpret the results:
        alpha = 0.05
        print(colored(p_value, "red"))
        if p_value < alpha:
            print(f"Reject the null hypothesis; there is a significant difference between {x} and {y}.")
        else:
            print(f"Fail to reject the null hypothesis; there is no significant difference between {x} and {y}.")